In [1]:
%pip install roboflow

  Using cached certifi-2026.1.4-py3-none-any.whl.metadata (2.5 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached matplotlib-3.10.8-cp311-cp311-macosx_11_0_arm64.whl.metadata (52 kB)
  Using cached numpy-2.4.2-cp311-cp311-macosx_14_0_arm64.whl.metadata (6.6 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached pillow-12.1.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (8.8 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached urllib3-2.6.3-py3-none-any.whl.metadata (6.9 kB)
  Using cached pyyaml-6.0.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.4 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached filetype-1.2.0

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="MYuIstjO07DOJx3bmDZW")
project = rf.workspace("farrellhrs").project("road-damage-detection-uutxu")
version = project.version(3)
dataset = version.download("yolov8", location="../data/road-damage-detection-yolov8-int")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to ../data/road-damage-detection-yolov8-int in yolov8:: 100%|██████████| 4564/4564 [00:00<00:00, 8342.50it/s]


In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="0CEdpB53poTT1jUXgvm7")
project = rf.workspace("rdd-heop9").project("road-damage-detection-ytvlw")
version = project.version(2)
dataset = version.download("yolov8", location="../data/road-damage-detection-yolov8-ext")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ../data/road-damage-detection-yolov8-ext in yolov8:: 100%|██████████| 2012/2012 [00:00<00:00, 5394.94it/s]


In [5]:
# Merge datasets

import os
import shutil
import yaml
from pathlib import Path

SRC1 = Path("../data/road-damage-detection-yolov8-int")
SRC2 = Path("../data/road-damage-detection-yolov8-ext")
DST  = Path("../data/road-damage-detection-yolov8-merged")

SPLITS = ["train", "valid", "test"]

def ensure_structure():
    for split in SPLITS:
        (DST / split / "images").mkdir(parents=True, exist_ok=True)
        (DST / split / "labels").mkdir(parents=True, exist_ok=True)

def copy_split(src, name_prefix):
    for split in SPLITS:
        img_src = src / split / "images"
        lbl_src = src / split / "labels"

        if not img_src.exists():
            continue

        for img in img_src.glob("*"):
            new_name = f"{name_prefix}_{img.name}"
            shutil.copy2(img, DST / split / "images" / new_name)

        for lbl in lbl_src.glob("*"):
            new_name = f"{name_prefix}_{lbl.name}"
            shutil.copy2(lbl, DST / split / "labels" / new_name)


def merge_yaml():
    yaml1 = yaml.safe_load(open(SRC1 / "data.yaml"))
    yaml2 = yaml.safe_load(open(SRC2 / "data.yaml"))

    # assume same class names but verify
    if yaml1["names"] != yaml2["names"]:
        raise ValueError("Class mismatch between datasets")

    merged = {
        "path": str(DST.resolve()),
        "train": "train/images",
        "val": "valid/images",
        "test": "test/images",
        "names": yaml1["names"],
        "nc": len(yaml1["names"]),
    }

    with open(DST / "data.yaml", "w") as f:
        yaml.dump(merged, f)

In [6]:
ensure_structure()

# prefix prevents filename collision
copy_split(SRC1, "int")
copy_split(SRC2, "ext")

merge_yaml()

print("Merged dataset ready at:", DST)

Merged dataset ready at: ../data/road-damage-detection-yolov8-merged
